# 1. Clean training data

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime

In [ ]:
pd.set_option('colwidth', 100)
pd.set_option('max_columns', 300)
pd.set_option('max_rows', 100)

In [ ]:
df= pd.read_csv('/Users/bonniechung/GoogleCloud/758T/project/raw_data/airbnb_train_x.csv', header = 0, index_col = 0, keep_default_na=False, dtype={"zipcode":str, 'maximum_nights':str, 'minimum_nights':str, 'availability_30':str, 'availability_60':str, 'availability_90':str, 'availability_365':str })


In [ ]:
df_y= pd.read_csv('/Users/bonniechung/GoogleCloud/758T/project/raw_data/airbnb_train_y.csv', header = 0, index_col = 0, keep_default_na=False)


In [ ]:
df = pd.concat([df_y,df], axis = 1)

In [ ]:
# Drop missing or invalid records
df.drop([16246,30584,47615,56281,65792,72540,75208,92585,96068], inplace = True)
df.drop([95973,548,75600,73072], inplace = True)
df.drop([13584, 24317, 29879, 44690, 51371, 63486, 76218, 85311, 90609,92028], inplace = True) # y wrong

In [ ]:
# Drop columns
df.drop(['require_guest_profile_picture'], axis = 1, inplace = True) # All of the values are false
df.drop(['square_feet'], axis = 1, inplace = True) # Having too less records
df.drop(['experiences_offered'], axis = 1, inplace = True)

In [ ]:
# Change datatype of y variables
df.iloc[:,0:2] = df.iloc[:,0:2].astype(int)

In [ ]:
# Calculate the length of text data
df['len_access']= df['access'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_description']= df['description'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_host_about']= df['host_about'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_house_rules']= df['house_rules'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_interaction']= df['interaction'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_name']= df['name'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_neighborhood_overview']= df['neighborhood_overview'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_notes']= df['notes'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_space']= df['space'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_summary']= df['summary'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_transit']= df['transit'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))

In [ ]:
df.drop(['description','host_about'], axis = 1, inplace = True)

In [ ]:
# Cast data into float type
df['availability_30']= df['availability_30'].apply(lambda x: float(x))
df['availability_60']= df['availability_60'].apply(lambda x: float(x))
df['availability_90']= df['availability_90'].apply(lambda x: float(x))
df['availability_365']= df['availability_365'].apply(lambda x: float(x))

In [ ]:
# Calculate average number for null data
def to_avg(c):
    a= np.average(df[df[c]!=''][c].astype(float))
    df[c]= df[c].replace('', a).astype(float)
to_avg('bathrooms')
to_avg('bedrooms')
to_avg('beds')
to_avg('host_listings_count')
to_avg('host_total_listings_count')

In [ ]:
# Set null data to 0
df['cleaning_fee']= df['cleaning_fee'].replace('', 0)
df['cleaning_fee']= df['cleaning_fee'].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
# Fill null data with 0 and set non-null data to 1
def nll(string):
    df[string]= df[string].notnull().astype('int')
nll('access')
nll('summary')
nll('space')
nll('notes')
nll('transit')
nll('host_name')
nll('house_rules')
nll('license')

In [ ]:
# Deal with geo data: get city and state from longitude and latitude, and clean zipcode, host_location, and neighborhood_overview
df.drop(['city','city_name','jurisdiction_names','neighbourhood','host_neighbourhood','street','state','country_code','country'], axis = 1, inplace = True)

# longitude, latitude
df['longitude']= df['longitude'].astype(float)
df['latitude']=df['latitude'].astype(float)

import reverse_geocoder as rg
ww=tuple(zip(df['latitude'],df['longitude']))
df['state']= np.array([i.get('admin1') for i in rg.search(ww)])
df['city']= np.array([i.get('name') for i in rg.search(ww)])

# zipcode
df['zipcode']= df['zipcode'].apply(lambda x: x.replace('.0','')).replace('\D', '', regex=True).apply(lambda x: x[:5])
df.loc[df['zipcode'].apply(lambda x: len(x))!=5, 'zipcode'] = '00000'

# host_location
df['host_location']=df['host_location'].notnull().astype('int')

#neighborhood_overview
df['neighborhood_overview']=df['neighborhood_overview'].notnull().astype('int')
df.drop(['smart_location'], axis = 1, inplace = True)

In [ ]:
# For cateforical data, set null as 'no_data'.
def to_no_data(c):
    df[c]= df[c].replace('', 'no_data')
to_no_data('host_response_time')
to_no_data('market')
to_no_data('property_type')
to_no_data('room_type')

In [ ]:
# Deal with date: convert date into days from now
# first_review
df['first_review']=df['first_review'].apply(lambda x: (dt.datetime.now() - datetime.strptime(x,'%Y-%m-%d')).days)
# host_since
df['host_since']=df['host_since'].apply(lambda x: (dt.datetime.now() - datetime.strptime(x,'%Y-%m-%d')).days if x else '')
to_avg('host_since')

In [ ]:
# Strip symbols
# extra_people
df['extra_people']= df['extra_people'].replace('[\$,]', '', regex=True).astype(float)

# host_acceptance_rate
df['host_acceptance_rate']= df['host_acceptance_rate'].replace('[%]', '', regex=True)
to_avg('host_acceptance_rate')

# host_response_rate
df['host_response_rate']= df['host_response_rate'].replace('[%]', '', regex=True)
to_avg('host_response_rate')

In [ ]:
# Change t, f into 1 and 0
def to_f(col):
    df[col]= df[col].replace(['f',''],'0').replace('t','1').astype(int)
to_f('require_guest_phone_verification')
to_f('requires_license')
to_f('instant_bookable')
to_f('is_location_exact')
to_f('is_business_travel_ready')
to_f('host_has_profile_pic')
to_f('host_is_superhost')
to_f('host_identity_verified')

In [ ]:
# Expend host_verifications into columns
a= df['host_verifications'].apply(lambda x: str(x))
a= a.apply(lambda x: x.replace('None','[]'))
a= a.apply(lambda x: x.replace('nan','[]'))
a= a.replace('','[]')
a= a.apply(lambda x: eval(x))
b = a.apply(frozenset).to_frame(name='genre')
for genre in frozenset.union(*b.genre):
    b[genre] = b.apply(lambda _: int(genre in _.genre), axis=1)
b=b.iloc[:,1:]
df= pd.concat([df, b], axis=1)
df.drop(['host_verifications'], axis = 1, inplace = True)

In [ ]:
# interaction
df['interaction']=df['interaction'].notnull().astype('int')

# maximum_nights
df['maximum_nights']=df['maximum_nights'].replace('NA','365', regex=True).astype(float)
df.loc[df['maximum_nights'] > 364, 'maximum_nights'] = 365

# minimum_nights
df['minimum_nights']=df['minimum_nights'].replace('NA','1', regex=True).astype(float)
df.loc[df['minimum_nights'] > 364, 'minimum_nights'] = 365

# monthly_price
df['monthly_price']= df['monthly_price'].replace('', '0')
df['monthly_price']= df['monthly_price'].replace('[\$,]', '', regex=True).astype(float)

# name
df.drop(['name'], axis = 1, inplace = True)

# price
df['price']= df['price'].replace('', '0')
df['price']= df['price'].replace('[\$,]', '', regex=True).astype(float)

# security_deposit
df['security_deposit']= df['security_deposit'].replace('', '0')
df['security_deposit']= df['security_deposit'].replace('[\$,]', '', regex=True).astype(float)

# weekly_price
df['weekly_price']= df['weekly_price'].replace('', '0')
df['weekly_price']= df['weekly_price'].replace('[\$,]', '', regex=True).astype(float)

df['weekly_discount']= df['price']*7-df['weekly_price']
df.drop(['weekly_price'], axis = 1, inplace = True)

df['monthly_discount']= df['price']*30-df['monthly_price']
df.drop(['monthly_price'], axis = 1, inplace = True)


In [ ]:
# Amenities
import re
a= df['amenities'].apply(lambda x : re.sub(r"[^,\w]+",'',x).lower())
a= a.apply(lambda x : x.split(','))
b= a.apply(frozenset).to_frame(name='genre')
for genre in frozenset.union(*b.genre):
    b[genre] = b.apply(lambda _: int(genre in _.genre), axis=1)

b['pets']= b['dogs'] + b['petsallowed'] + b['cats'] + b['otherpets']
b['pets']= b['pets'].replace([2,3,4],1)

b['tv']= b['tv'] + b['cabletv'] + b['smarttv']
b['tv']= b['tv'].replace([2,3],1)

b['elevator']= b['elevator'] + b['elevatorinbuilding']
b['elevator']= b['elevator'].replace(2,1)

b['freeparkingonstreet']= b['freeparkingonstreet'] + b['freestreetparking']
b['freestreetparking']= b['freeparkingonstreet'].replace(2,1)

b['frontdesk/doorperson']= b['frontdeskdoorperson'] + b['doorman']
b['frontdesk/doorperson']= b['frontdesk/doorperson'].replace(2,1)

b['wifi']= b['wifi'] + b['wirelessinternet']
b['wifi']= b['wifi'].replace(2,1)

b['oven']= b['oven'] + b['convectionoven']
b['oven']= b['oven'].replace(2,1)

b['widedoorway']= b['widedoorway'] + b['wideentryway']
b['widedoorway']= b['widedoorway'].replace(2,1)

b['wideclearancetoshowerandtoilet']= b['wideclearancetoshowerandtoilet'] + b['wideclearancetoshowertoilet']
b['wideclearancetoshowerandtoilet']= b['wideclearancetoshowerandtoilet'].replace(2,1)

b['welllitpathtoentrance']= b['welllitpathtoentrance'] + b['pathtoentrancelitatnight']
b['lit_entrance']= b['welllitpathtoentrance'].replace(2,1)

b['balcony']= b['balcony'] + b['patioorbalcony']
b['balcony']= b['balcony'].replace(2,1)

b['bathtub']= b['bathtub'] + b['soakingtub']
b['bathtub']= b['bathtub'].replace(2,1)

b['kitchen']= b['kitchen'] + b['kitchenette'] + b['fullkitchen']
b['kitchen']= b['kitchen'].replace([2,3,4],1)

b['flatpathtofrontdoor']= b['flatpathtofrontdoor'] + b['flatsmoothpathwaytofrontdoor'] + b['flatpathtofrontdoor'] + b['smoothpathwaytofrontdoor']
b['flatpathtofrontdoor']= b['flatpathtofrontdoor'].replace([2,3,4],1)

b['fixedgrabbarsforshower']= b['fixedgrabbarsforshower'] + b['fixedgrabbarsforshowertoilet']
b['fixedgrabbarsforshower']= b['fixedgrabbarsforshower'].replace(2,1)

b.drop(['genre', '', 'dogs', 'petsallowed', 'cats', 'otherpets','cabletv','smarttv','elevatorinbuilding', 'freeparkingonstreet','frontdeskdoorperson', 'doorman', 'wirelessinternet', 'convectionoven','fixedgrabbarsforshowertoilet', 'flatsmoothpathwaytofrontdoor', 'flatpathtofrontdoor', 'smoothpathwaytofrontdoor', 'kitchenette', 'fullkitchen', 'soakingtub', 'patioorbalcony', 'welllitpathtoentrance', 'pathtoentrancelitatnight', 'wideclearancetoshowertoilet', 'wideentryway'], axis = 1, inplace = True)
df.drop(['amenities'], axis = 1, inplace = True)

df = pd.concat([df,b], axis = 1)


In [ ]:
# Expand categories into columns
def dum(col):
    global df
    a= pd.get_dummies(df[col].str.lower(),prefix=col)
    df= pd.concat([df,a], axis = 1)
    df= df.drop([col], axis = 1)
dum('bed_type')
dum('cancellation_policy')
dum('host_response_time')
dum('market')
dum('property_type')
dum('room_type')
dum('city')
dum('state')

In [ ]:
# Save to file
df.to_csv('airbnb_train.csv')

# 2. Clean testing data

In [ ]:
df= pd.read_csv('/Users/bonniechung/GoogleCloud/758T/project/raw_data/airbnb_test_x.csv', header = 0, index_col = 0, keep_default_na=False, dtype={"zipcode":str, 'maximum_nights':str, 'minimum_nights':str, 'availability_30':str, 'availability_60':str, 'availability_90':str, 'availability_365':str })


In [ ]:
# Drop missing or invalid records
df.drop([775,10274], inplace = True)

In [ ]:
# Drop columns
df.drop(['require_guest_profile_picture'], axis = 1, inplace = True) # All of the values are false
df.drop(['square_feet'], axis = 1, inplace = True) # Having too less records
df.drop(['experiences_offered'], axis = 1, inplace = True)

In [ ]:
# Calculate the length of text data
df['len_access']= df['access'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_description']= df['description'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_host_about']= df['host_about'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_house_rules']= df['house_rules'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_interaction']= df['interaction'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_name']= df['name'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_neighborhood_overview']= df['neighborhood_overview'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_notes']= df['notes'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_space']= df['space'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_summary']= df['summary'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))
df['len_transit']= df['transit'].replace("[^a-zA-Z0-9' ]", '',regex=True).apply(lambda x:len(x))

In [ ]:
df.drop(['description','host_about'], axis = 1, inplace = True)

In [ ]:
# Cast data into float type
df['availability_30']= df['availability_30'].apply(lambda x: float(x))
df['availability_60']= df['availability_60'].apply(lambda x: float(x))
df['availability_90']= df['availability_90'].apply(lambda x: float(x))
df['availability_365']= df['availability_365'].apply(lambda x: float(x))

In [ ]:
# Calculate average number for null data
def to_avg(c):
    a= np.average(df[df[c]!=''][c].astype(float))
    df[c]= df[c].replace('', a).astype(float)
to_avg('bathrooms')
to_avg('bedrooms')
to_avg('beds')
to_avg('host_listings_count')
to_avg('host_total_listings_count')

In [ ]:
# Set null data to 0
df['cleaning_fee']= df['cleaning_fee'].replace('', 0)
df['cleaning_fee']= df['cleaning_fee'].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
# Fill null data with 0 and set non-null data to 1
def nll(string):
    df[string]= df[string].notnull().astype('int')
nll('access')
nll('summary')
nll('space')
nll('notes')
nll('transit')
nll('host_name')
nll('house_rules')
nll('license')

In [ ]:
# Deal with geo data: get city and state from longitude and latitude, and clean zipcode, host_location, and neighborhood_overview
df.drop(['city','city_name','jurisdiction_names','neighbourhood','host_neighbourhood','street','state','country_code','country'], axis = 1, inplace = True)

# longitude, latitude
df['longitude']= df['longitude'].replace('',0, regex=True).astype(float)
df['latitude']=df['latitude'].replace('',0, regex=True).astype(float)

import reverse_geocoder as rg
ww=tuple(zip(df['latitude'],df['longitude']))
df['state']= np.array([i.get('admin1') for i in rg.search(ww)])
df['city']= np.array([i.get('name') for i in rg.search(ww)])

# zipcode
df['zipcode']= df['zipcode'].apply(lambda x: x.replace('.0','')).replace('\D', '', regex=True).apply(lambda x: x[:5])
df.loc[df['zipcode'].apply(lambda x: len(x))!=5, 'zipcode'] = '00000'

# host_location
df['host_location']=df['host_location'].notnull().astype('int')

#neighborhood_overview
df['neighborhood_overview']=df['neighborhood_overview'].notnull().astype('int')
df.drop(['smart_location'], axis = 1, inplace = True)

In [ ]:
# For cateforical data, set null as 'no_data'.
def to_no_data(c):
    df[c]= df[c].replace('', 'no_data')
to_no_data('host_response_time')
to_no_data('market')
to_no_data('property_type')
to_no_data('room_type')

In [ ]:
# Deal with date: convert date into days from now
# first_review
df['first_review']=df['first_review'].apply(lambda x: (dt.datetime.now() - datetime.strptime(x,'%Y-%m-%d')).days)
# host_since
df['host_since']=df['host_since'].apply(lambda x: (dt.datetime.now() - datetime.strptime(x,'%Y-%m-%d')).days if x else '')
to_avg('host_since')

In [ ]:
# Strip symbols
# extra_people
df['extra_people']= df['extra_people'].replace('[\$,]', '', regex=True).astype(float)

# host_acceptance_rate
df['host_acceptance_rate']= df['host_acceptance_rate'].replace('[%]', '', regex=True)
to_avg('host_acceptance_rate')

# host_response_rate
df['host_response_rate']= df['host_response_rate'].replace('[%]', '', regex=True)
to_avg('host_response_rate')

In [ ]:
# Change t, f into 1 and 0
def to_f(col):
    df[col]= df[col].replace(['f',''],'0').replace('t','1').astype(int)
to_f('require_guest_phone_verification')
to_f('requires_license')
to_f('instant_bookable')
to_f('is_location_exact')
to_f('is_business_travel_ready')
to_f('host_has_profile_pic')
to_f('host_is_superhost')
to_f('host_identity_verified')

In [ ]:
# Expend host_verifications into columns
a= df['host_verifications'].apply(lambda x: str(x))
a= a.apply(lambda x: x.replace('None','[]'))
a= a.apply(lambda x: x.replace('nan','[]'))
a= a.replace('','[]')
a= a.apply(lambda x: eval(x))
b = a.apply(frozenset).to_frame(name='genre')
for genre in frozenset.union(*b.genre):
    b[genre] = b.apply(lambda _: int(genre in _.genre), axis=1)
b=b.iloc[:,1:]
df= pd.concat([df, b], axis=1)
df.drop(['host_verifications'], axis = 1, inplace = True)

In [ ]:
# interaction
df['interaction']=df['interaction'].notnull().astype('int')

# maximum_nights
df['maximum_nights']=df['maximum_nights'].replace('NA','365', regex=True).astype(float)
df.loc[df['maximum_nights'] > 364, 'maximum_nights'] = 365

# minimum_nights
df['minimum_nights']=df['minimum_nights'].replace('NA','1', regex=True).astype(float)
df.loc[df['minimum_nights'] > 364, 'minimum_nights'] = 365

# monthly_price
df['monthly_price']= df['monthly_price'].replace('', '0')
df['monthly_price']= df['monthly_price'].replace('[\$,]', '', regex=True).astype(float)

# name
df.drop(['name'], axis = 1, inplace = True)

# price
df['price']= df['price'].replace('', '0')
df['price']= df['price'].replace('[\$,]', '', regex=True).astype(float)

# security_deposit
df['security_deposit']= df['security_deposit'].replace('', '0')
df['security_deposit']= df['security_deposit'].replace('[\$,]', '', regex=True).astype(float)

# weekly_price
df['weekly_price']= df['weekly_price'].replace('', '0')
df['weekly_price']= df['weekly_price'].replace('[\$,]', '', regex=True).astype(float)

df['weekly_discount']= df['price']*7-df['weekly_price']
df.drop(['weekly_price'], axis = 1, inplace = True)

df['monthly_discount']= df['price']*30-df['monthly_price']
df.drop(['monthly_price'], axis = 1, inplace = True)


In [ ]:
# Amenities
import re
a= df['amenities'].apply(lambda x : re.sub(r"[^,\w]+",'',x).lower())
a= a.apply(lambda x : x.split(','))
b= a.apply(frozenset).to_frame(name='genre')
for genre in frozenset.union(*b.genre):
    b[genre] = b.apply(lambda _: int(genre in _.genre), axis=1)

b['pets']= b['dogs'] + b['petsallowed'] + b['cats'] + b['otherpets']
b['pets']= b['pets'].replace([2,3,4],1)

b['tv']= b['tv'] + b['cabletv'] + b['smarttv']
b['tv']= b['tv'].replace([2,3],1)

b['elevator']= b['elevator'] + b['elevatorinbuilding']
b['elevator']= b['elevator'].replace(2,1)

b['freeparkingonstreet']= b['freeparkingonstreet'] + b['freestreetparking']
b['freestreetparking']= b['freeparkingonstreet'].replace(2,1)

b['frontdesk/doorperson']= b['frontdeskdoorperson'] + b['doorman']
b['frontdesk/doorperson']= b['frontdesk/doorperson'].replace(2,1)

b['wifi']= b['wifi'] + b['wirelessinternet']
b['wifi']= b['wifi'].replace(2,1)

b['oven']= b['oven'] + b['convectionoven']
b['oven']= b['oven'].replace(2,1)

b['widedoorway']= b['widedoorway'] + b['wideentryway']
b['widedoorway']= b['widedoorway'].replace(2,1)

b['welllitpathtoentrance']= b['welllitpathtoentrance'] + b['pathtoentrancelitatnight']
b['lit_entrance']= b['welllitpathtoentrance'].replace(2,1)

b['balcony']= b['balcony'] + b['patioorbalcony']
b['balcony']= b['balcony'].replace(2,1)

b['bathtub']= b['bathtub'] + b['soakingtub']
b['bathtub']= b['bathtub'].replace(2,1)

b['kitchen']= b['kitchen'] + b['fullkitchen']
b['kitchen']= b['kitchen'].replace([2,3,4],1)

b['flatpathtofrontdoor']= b['flatpathtofrontdoor'] + b['flatpathtofrontdoor'] + b['smoothpathwaytofrontdoor']
b['flatpathtofrontdoor']= b['flatpathtofrontdoor'].replace([2,3,4],1)

b['fixedgrabbarsforshower']= b['fixedgrabbarsforshower'] + b['fixedgrabbarsforshowertoilet']
b['fixedgrabbarsforshower']= b['fixedgrabbarsforshower'].replace(2,1)

b.drop(['genre', '', 'dogs', 'petsallowed', 'cats', 'otherpets','cabletv','smarttv','elevatorinbuilding', 'freeparkingonstreet','frontdeskdoorperson', 'doorman', 'wirelessinternet', 'convectionoven','fixedgrabbarsforshowertoilet', 'flatpathtofrontdoor', 'smoothpathwaytofrontdoor', 'fullkitchen', 'soakingtub', 'patioorbalcony', 'welllitpathtoentrance', 'pathtoentrancelitatnight', 'wideclearancetoshowertoilet', 'wideentryway'], axis = 1, inplace = True)
df.drop(['amenities'], axis = 1, inplace = True)

df = pd.concat([df,b], axis = 1)


In [ ]:
# Expand categories into columns
def dum(col):
    global df
    a= pd.get_dummies(df[col].str.lower(),prefix=col)
    df= pd.concat([df,a], axis = 1)
    df= df.drop([col], axis = 1)
dum('bed_type')
dum('cancellation_policy')
dum('host_response_time')
dum('market')
dum('property_type')
dum('room_type')
dum('city')
dum('state')

In [ ]:
t= pd.read_csv('airbnb_train.csv', index_col = 0)

In [ ]:
for i in t.keys():
    if i not in df.keys():
        df[str(i)]=np.zeros(len(df), dtype=int)

In [ ]:
df= df[list(t.keys())]

In [ ]:
# Save to file
df.to_csv('airbnb_test.csv')

# 3. Build models

## 3-1 Prepare data

In [ ]:
df= pd.read_csv('airbnb_train.csv', header = 0, index_col = 0)
df_t= pd.read_csv('airbnb_test.csv', header = 0, index_col = 0)
df_g = pd.read_csv('/Users/bonniechung/GoogleCloud/758T/project/raw_raw/airbnb_geo.csv',encoding='utf-8',index_col = 0)

In [ ]:
a= (df[['longitude','latitude']].round(6)*1000000).astype(int)
a= a.reset_index()
b= df_g[df_g['longitude'].notnull()][['longitude','latitude','review_scores_accuracy','reviews_per_month','review_scores_communication',
                                      'review_scores_cleanliness','review_scores_location','review_scores_rating'
                                      ,'review_scores_value','number_of_reviews']]
b.columns=['longitude','latitude','review_scores_accuracy','reviews_per_month','review_scores_communication',
                                      'review_scores_cleanliness','review_scores_location','review_scores_rating2'
                                      ,'review_scores_value','number_of_reviews']
b[['longitude','latitude']]=(b[['longitude','latitude']].round(6)*1000000).astype(int)
a[a[['longitude','latitude']].duplicated()]

In [ ]:
b[b[['longitude','latitude']].duplicated()]

In [ ]:
c= pd.merge(a,b, how='left',on=['longitude','latitude'])
c= c.set_index('index')
del c.index.name
c.drop(['longitude','latitude'], axis = 1, inplace = True)
df= pd.concat([df,c],axis=1)
df[['review_scores_accuracy','reviews_per_month','review_scores_communication', 'review_scores_cleanliness',
    'review_scores_location','review_scores_rating2','review_scores_value',
    'number_of_reviews']]= df[['review_scores_accuracy','reviews_per_month',
                               'review_scores_communication','review_scores_cleanliness','review_scores_location',
                               'review_scores_rating2','review_scores_value','number_of_reviews']].fillna(0).astype(float)

In [ ]:
a= (df_t[['longitude','latitude']].round(6)*1000000).astype(int)
a= a.reset_index()
a[a[['longitude','latitude']].duplicated()]

In [ ]:
c= pd.merge(a,b, how='left',on=['longitude','latitude'])
c= c.set_index('index')
del c.index.name
c.drop(['longitude','latitude'], axis = 1, inplace = True)
df_t= pd.concat([df_t,c],axis=1)
df_t[['review_scores_accuracy','reviews_per_month','review_scores_communication', 'review_scores_cleanliness',
    'review_scores_location','review_scores_rating2','review_scores_value',
    'number_of_reviews']]= df_t[['review_scores_accuracy','reviews_per_month',
                               'review_scores_communication','review_scores_cleanliness','review_scores_location',
                               'review_scores_rating2','review_scores_value','number_of_reviews']].fillna(0).astype(float)

### 3-1-1 Scale data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Training data
a = StandardScaler().fit_transform(df.iloc[:,2:].values)
b = pd.DataFrame(a, index=df.iloc[:,2:].index, columns=df.iloc[:,2:].columns)
df = pd.concat([df.iloc[:,:2],b],axis=1)

In [ ]:
# Testing data
a = StandardScaler().fit_transform(df_t.values)
df_t = pd.DataFrame(a, index=df_t.index, columns=df_t.columns)

## 3-2 Split data into training and testing data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(df.iloc[:,2:], df.iloc[:,1], test_size=0.2)

## 3-3 XGBClassifier

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [ ]:
def modelfit(alg, train_x=train_x ,train_y=train_y, test_x=test_x, test_y=test_y, 
             useTrainCV=True, cv_folds=5, early_stopping_rounds=100):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds,verbose_eval=True)
        alg.set_params(n_estimators=cvresult.shape[0])
        
    
    #Fit the algorithm on the data
    alg.fit(train_x, train_y,eval_metric='auc')
    print(alg)
        
    #Predict training set:
    train_predictions = alg.predict(train_x)
    train_predprob = alg.predict_proba(train_x)[:,1]

    #Predict testing set:
    test_predictions = alg.predict(test_x)
    test_predprob = alg.predict_proba(test_x)[:,1]
    
    #Print model report:
    print("\nModel Report")
    print("Accuracy (Train): %.4g" % metrics.accuracy_score(train_y.values, train_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(train_y, train_predprob))
    print("Accuracy (Test): %.4g" % metrics.accuracy_score(test_y.values, test_predictions))
    print("AUC Score (Test): %f" % metrics.roc_auc_score(test_y, test_predprob))

    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False).head(30)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
# Get best n_estimator with cross-validation
xgb1 = XGBClassifier(n_estimators=10000,n_jobs=3,seed=2 )
modelfit(xgb1)

### 3-3-1 Tune Parameters

In [ ]:
param_test1 = {
 'max_depth':list(range(3,6,2)),
 'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=2362, max_depth=5,min_child_weight=1, 
                                    gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                                    n_jobs=3, scale_pos_weight=1, seed=2), 
                        param_grid = param_test1, scoring='roc_auc',n_jobs=2,iid=False, cv=5)
gsearch1.fit(train_x,train_y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test2 = {
 'max_depth':[2,3,4],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=2362, max_depth=3,
                                    min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                    objective= 'binary:logistic', n_jobs=3, scale_pos_weight=1,seed=2), 
                         param_grid = param_test2, scoring='roc_auc',n_jobs=2,iid=False, cv=5)
gsearch2.fit(train_x,train_y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test2b = {
 'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=2362, max_depth=4,
                                     min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                     objective= 'binary:logistic', n_jobs=3, scale_pos_weight=1,seed=2), 
                         param_grid = param_test2b, scoring='roc_auc',n_jobs=2,iid=False, cv=5)
gsearch2b.fit(train_x,train_y)
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=3000, max_depth=4,
                                    min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                    objective= 'binary:logistic', n_jobs=3, scale_pos_weight=1,seed=2), 
                        param_grid = param_test3, scoring='roc_auc',n_jobs=2,iid=False, cv=5)
gsearch3.fit(train_x,train_y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=2362, max_depth=4,
                                    min_child_weight=6, gamma=0.3, subsample=0.8, colsample_bytree=0.8,
                                    objective= 'binary:logistic', n_jobs=3, scale_pos_weight=1,seed=2), 
                        param_grid = param_test4, scoring='roc_auc',n_jobs=2,iid=False, cv=5)
gsearch4.fit(train_x,train_y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

In [ ]:
param_test5 = {
 'subsample':[i/100.0 for i in range(75,90,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=2362, max_depth=4,
                                    min_child_weight=6, gamma=0.3, subsample=0.8, colsample_bytree=0.8,
                                    objective= 'binary:logistic', n_jobs=3, scale_pos_weight=1,seed=2), 
                        param_grid = param_test5, scoring='roc_auc',n_jobs=2,iid=False, cv=5)
gsearch5.fit(train_x,train_y)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

In [ ]:
param_test6 = {
 'reg_alpha':[1e-6, 1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=2362, max_depth=4,
                                    min_child_weight=6, gamma=0.3, subsample=0.85, colsample_bytree=0.85,
                                    objective= 'binary:logistic', n_jobs=3, scale_pos_weight=1,seed=2), 
                        param_grid = param_test6, scoring='roc_auc',n_jobs=2,iid=False, cv=5)
gsearch6.fit(train_x,train_y)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

In [ ]:
param_test7 = {
 'reg_alpha':list(np.arange(1e-06,1e-04,0.00002))
}
gsearch7 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=2362, max_depth=4,
                                    min_child_weight=6, gamma=0.3, subsample=0.85, colsample_bytree=0.85,
                                    objective= 'binary:logistic', n_jobs=3, scale_pos_weight=1,seed=2), 
                        param_grid = param_test7, scoring='roc_auc',n_jobs=2,iid=False, cv=5)
gsearch7.fit(train_x,train_y)
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

In [ ]:
xgb_tune = XGBClassifier(learning_rate =0.1, n_estimators=2362, max_depth=4,min_child_weight=6, gamma=0.3, 
                     subsample=0.85, colsample_bytree=0.85,objective= 'binary:logistic', n_jobs=3, 
                     scale_pos_weight=1,seed=2)
modelfit(xgb_tune)

## 3-4 Use all training data to build the model

In [ ]:
train_x = df.iloc[:,2:]
train_y = df.iloc[:,1]
test_x  = df_t.iloc[:,2:]

In [ ]:
xgb2 = XGBClassifier(n_estimators=4000,n_jobs=3,seed=2)
xgb_param = xgb2.get_xgb_params()
xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=xgb2.get_params()['n_estimators'], 
                  nfold=5,metrics='auc', early_stopping_rounds=150)
xgb2.set_params(n_estimators=cvresult.shape[0])
xgb2.fit(train_x, train_y,eval_metric='auc')

## 3-5 Fit testing data into the model and output the results

In [ ]:
predictions = xgb2.predict(test_x)
predictions= list(predictions)
predictions.insert(774,0)
predictions.insert(10273,0)

In [ ]:
pd.DataFrame(predictions, columns=['high_booking_rate'],index=np.arange(1,12209)).to_csv('raw_0427.csv')